In [1]:
# !pip install tabpfn

# ЖКХ

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd 
import joblib

from sklearn.preprocessing import LabelEncoder


data = pd.read_csv('fortab')
features = data.drop('Тип', axis=1)  # Замените 'target_column' на имя вашего столбца с метками
features = features[features.columns[3:]]
mean_values = features.median()
features.fillna(mean_values, inplace=True)
features = features.fillna('', downcast='infer')

# label_encoder = LabelEncoder()

# # Iterate through string columns and apply label encoding
# for column in features.select_dtypes(include=['object']):
#     features[column] = label_encoder.fit_transform(features[column])


# # Identify columns with object (string) data types
# string_columns = features.select_dtypes(include=['object']).columns

# # Use get_dummies to one-hot encode identified columns
# features = pd.get_dummies(features, columns=string_columns)
features = features.select_dtypes(exclude=['object'])
print(features.shape)
print(features)

target = data['Тип']

# print(features.head())
# print(target.head())
features.to_csv('GKH.csv')

(1477, 28)
      Сетей теплоснабжения  Сетей холодного водоснабжения  \
0                     77.0                           20.0   
1                     77.0                           20.0   
2                     77.0                           20.0   
3                     77.0                           20.0   
4                     77.0                           20.0   
...                    ...                            ...   
1472                  57.0                           75.0   
1473                  57.0                           75.0   
1474                  57.0                           75.0   
1475                  57.0                           75.0   
1476                  57.0                           75.0   

      Сетей горячего водоснабжения  Сетей электроснабжения  \
0                               69                       0   
1                               69                       0   
2                               69                       0   
3       

/tmp/ipykernel_92568/2570831483.py:11: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values = features.median()


In [3]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


# X_train = pd.read_csv('X_train.csv')
# X_test = pd.read_csv('X_test.csv')

# y_train = pd.read_csv('y_train.csv')
# y_test = pd.read_csv('y_test.csv')

# y_train = y_train[[*y_train.columns[[4]]]]
# y_test = y_test[[*y_test.columns[[4]]]]

In [4]:
%%time

from tabpfn import TabPFNClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.multioutput import MultiOutputClassifier

classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)
# classifier = MultiOutputClassifier(classifier)

classifier.fit(X_train, y_train, overwrite_warning=True)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)
# print(len(set(ey_eval)), len(y_test.unique()))
print('Accuracy', accuracy_score(y_test, y_eval))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy 0.7972972972972973
CPU times: user 1min 4s, sys: 14.9 s, total: 1min 19s
Wall time: 12.4 s


In [5]:
# Save model
model_name = f"GKH.joblib"
joblib.dump(classifier, model_name)

['GKH.joblib']

In [6]:
y_eval[0]

'Аварии на электроэнергетических системах '

In [7]:
model_gkh = joblib.load("GKH.joblib")
y_preds, yy = model_gkh.predict(X_test, return_winning_probability=True)
y_preds

/home/eresque/.local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


array(['Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на коммунальных системах жизнеобеспечения Теплоснабжение',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на электроэнергетических системах ',
       'Аварии на коммунальных системах жизнеобеспечения Теплоснабжение',
       'Аварии на эл